### Background

If you pay much attention to college basketball, and even further, if you enjoy nerding out on stats, you probably know just how many different rating systems there are. RPI, Sagarin, Kenpom, Massey, etc... There are a lot of rating systems created by a lot of smart people, some complicated, some simple. 

There is plenty of motivation to come up with accurate rating systems for college basketball:

* Money - Rating systems can be used to predict the outcomes of games. Among other things, this can be used to make money. I won't say how.
* Bragging rights - College basketball fans are a competitive bunch and the ability to point to a concrete number that says "my team is better than yours" is just plain necessary.
* Interest - Rating teams on overall ability, or really any other aspect of their game, is just plain interesting. It helps to put real, concrete numbers to things fans see (or don't see) with their eyes when they watch the game.

### Why college basketball ratings are so hard

One of the most natural and easy ways to rate teams, in any sport, is to look at the ratio of games won to games lost, aka their win-loss record. In some sports this works well, and in other sports it doesn't really work at all. Why? The main reason is that it fails to account for **_strength of schedule_**.

There are other reasons too, of course, but in college basketball there is a huge disparity between the top teams and the bottom teams, one that really doesn't exist in other major American sports. There are 351 NCAA Division 1 college basketball teams. They can't all play each other and what ends up happening is that some teams play a schedule consisting of mostly bottom tier opponents, while others play schedules consisting of the top tier. 

We can't say that because Northwest Texas-Corpus Christi Polytechnic went 29-2 in the Mideastern Southwestern Atlantic Gulf Coast Conference (MESWAGCC) that they are as good as a Kansas team that went 29-2 in the Big 12. 

## Kenpom ratings

### Preface
Kenpom ratings measure a team's offensive and defensive output in terms of _points per possession_. This statistic just tells us how many points a team scores per opportunity it has to score. Looking at raw game scores doesn't make any sense at all; if some team scores 70 points in 50 possessions, we don't say their offense did worse than a team that scored 71 points in 80 possessions. We adjust for tempo.

### The ratings
Let's start at a very high level and think about how we can rate a team's offense. For a given team, we want to figure out how many points per possession that team would score against an average defense. If we know this for each team, we can use it as a way to rate and rank a team's offense. The good team's will have an offensive rating that says they score more against an average defense and the bad team's will have a rating that says they score less. We will seek the same result for each team's defense in a similar manner.

Let's start with the good - we know exactly how many points per possession a team scores in its each of its games from the box score. If each team plays a perfectly average defense in every game, then we could just average all of their point per possession outputs and call it a day. The problem is that they aren't playing an average defense night in and night out - over the course of a season a team will face some very _above_ average and some very _below_ average defenses. So, we have a problem.

### Let's adjust

Kenpom deals with this problem by _adjusting_ each team's raw offensive output up or down, depending on how good the defense in each game was. We adjust the raw output by multiplying it by some factor that measures the quality of the opponent. If the factor is greater than one, then we are increasing the raw output, and saying that the defense they faced was pretty good! If the number is less than one, we say that they faced a pretty poor defense and so the raw number was a bit inflated to begin with.

How do we find that ratio? Well, for a particular game, we just take the offensive output for an average team and divide it by the amount of points per possession that the defense allows to an average team (which is exactly the defensive rating of the opposing team!). That looks like this: 

$$
\text{adjustment_factor} = \frac{{ppp}_{avg}}{\tilde{d}^j} \\
\tilde{d}^j - \text{adjusted defensive rating for the jth team}
$$

All we really asdfneed to do now is to extend this idea to incorporate all of a team's games. How do we compute a single adjusted rating for a team, by including results from all of its games? The simplest way is to just average the adjusted points per possession outputs for each game. In maths, that looks like: 

$$
\tilde{o}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} {ppp}^j_i \left(\frac{{ppp}_{avg}}{\tilde{d}_i^j}\right) \\
N_j - \text{number of games played by team j}
$$

Alright, this is starting to shape up. We have a formula! That formula consists the following:

* $N_j$ - the number of games played by team j (we know this!)
* ${ppp}^j_i$ - the points per possession scored by team j in their ith game (we know it from the box score!)
* ${ppp}_{avg}$ - the average points per possession scored by any team in the league (we know it by averaging ${ppp}$ for all the games!)
* $\tilde{d}^j$ - the adjusted defensive rating for team j (uhhh...???)

Well, shit. We don't know one of the terms in our equation. No worries, we can remedy this by simply computing the adjusted defensive rating for each team. We can just use the same logic we used for offensive adjusted ratings to find a formula for defense. For defense, we have:


\tilde{d}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} {dppp}^j_i \left(\frac{{dppp}_{avg}}{\tilde{o}_i^j}\right) \\
{dppp}^j_i - \text{the points per possession allowed by team j in their ith game}


Again, we have an equation where we know everything but the denominator of the adjustment factor, in this case the adjusted offensive rating for each team. So, we really haven't gotten anywhere. In order to compute the adjusted offensive ratings we need the adjusted defensive ratings, but in order to compute those, we need the adjusted offensive ratings! How can we escape this circular hell?

### Iterate

It turns out, that we can actually create an [iterative method](https://en.wikipedia.org/wiki/Iterative_method) to solve this problem. Basically, we can start out by guessing an adjusted defensive rating for each team. Now that we have actual numbers for adjusted defense, we can compute adjusted offensive ratings. They won't be correct, of course, because we didn't use correct values for adjusted defense. The key here is that, although they won't be correct, they **_will_** be closer than they were. That is, on the whole, the offensive ratings will get adjusted in the right directions. And now that we have just computed a new, more intelligent value for adjusted offense, we can use those to compute new, more intelligent adjusted defensive ratings.

Every time we calculate new adjusted offensive and defensive ratings, they get closer to the true values. Eventually, they stop changing because they have reached the true values and we have our answer. Pretty amazing shit, eh?

Using some python code I wrote to compute Kenpom style adjusted stats, we can actually see this in action. I will go more into the code in a future post, but for now we can see a plot of the algorithm converging. We'll use the toy dataset below to compute adjusted ratings for each team.

<img src="dataframe.png" width=400>

**Note**: "hppp" and "appp" are the home and away points per possession for each game.

When we run the algorithm, we make an initial guess at the value of each team's adjusted ratings. In this case, the initial guess is just a reasonably select constant number and is used as each team's guess. We then see how the algorithm updates the guesses each time it iterates, and eventually reaches the true answers.

<img src="iter_plot.png" width=600>

Notice that the values stop changing after about six iterations, so there is really no need to run it more than this.

Now that we are equipped with our super-cool iterative algorithm and super-fancy formulas, we can compute our adjusted offensive and adjusted defensive ratings for each team. If these turn out to be any good, we can then use them to predict the winner of a game before it ever happens. And if that turns out to work well, we profit.

### Gotchas and caveats

It's not all gravy from this point, unfortunately. There are a few issues that the keen reader will have noticed with our formula. 
1. First, we haven't adjusted for home court advantage, which is particulary obscene given that no other sport has such a pronounced home court advantage as NCAA Division I college basketball. Fortunately for us, adjusting our formula to take this into account is fairly simple. We can just adjust each game by a home factor. If a team played at home we lower their offensive output, and if they played an away game we increase it. That leaves us with the formula:

\tilde{o}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} {ppp}^j_i \left(\frac{{ppp}_{avg}}{\tilde{d}_i^j}\right) \left(\frac{1}{loc}\right) \\
\tilde{d}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} {dppp}^j_i \left(\frac{{dppp}_{avg}}{\tilde{o}_i^j}\right) \left({loc}\right)


2. Simply taking an average of the adjusted outputs might not be the best idea. To illustrate this point, consider late in the season when a team may have played upwards of 25 games. In the current framework, the algorithm gives the same weight to the first team the game played as it does to the most recent game. Even though the team, late in the season, is likely very different than it was at the start. To account for this, we simply take a weighted average (instead of a uniform average), where we are free to make the weights whatever we want. This way, we can make the weights earlier in the season die out and the adjusted ratings will better reflect a team's recent performance. Our new formulas are:

\tilde{o}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} {ppp}^j_i \left(\frac{{ppp}_{avg}}{\tilde{d}_i^j}\right) \cdot \left(\frac{1}{loc}\right) \cdot w_i \\
\tilde{d}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} {dppp}^j_i \left(\frac{{dppp}_{avg}}{\tilde{o}_i^j}\right) \cdot {loc} \cdot w_i


3. Finally, we would like to rate teams when few or zero games have been played. Moreover, when only a few games have been played by a team, we do not want to be forced to rate them on such a small sample size. To accomplish this, we can come up with a preseason rating for each team. If that preseason rating is intelligent, then we will be able to rely on it until each team has played enough games to give us an idea of their true skill. We can use the weights to give more emphasis to the preseason at the start of the season, and then gradually fade out its effect as games are played. Our final formulas are then:

\tilde{o}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} \left[{ppp}^j_i \left(\frac{{ppp}_{avg}}{\tilde{d}_i^j}\right) \cdot \left(\frac{1}{loc}\right) \cdot w_i \right] + w_{pre} \tilde{o}_{pre, j}\\
\tilde{d}_j = \frac{1}{N_j} \sum_{i=1}^{N_j} \left[{dppp}^j_i \left(\frac{{dppp}_{avg}}{\tilde{o}_i^j}\right) \cdot {loc} \cdot w_i \right] + w_{pre} \tilde{d}_{pre, j}

